# centerflow

**centerflow** is a Python module for modeling glacier dynamics along flowlines. It provides tools with four core functionalities:

1. **Mesh generation**  
   Construct a 1D finite element mesh along an RGI-defined glacier centerline.

2. **Data interpolation**  
   Interpolate gridded geospatial datasets (e.g., surface elevation, velocity, surface mass balance) onto the centerline mesh.

3. **Data re-interpolation**  
   Extend interpolated functions onto a longer mesh, providing a buffer for terminus advance within mesh bounds. 

4. **Bed inversion**  
   Apply a forward-model-based bed inversion scheme following the approach of [van Pelt at al. (2013)](https://tc.copernicus.org/articles/7/987/2013/), using observed surface elevations to iteratively estimate basal topography.

These tools are designed for efficient, reproducible glacier modeling along flowlines.

## Imports

In [ ]:
from dataclasses import dataclass
import firedrake
import geopandas as gpd
import icepack
import numpy as np
import pandas as pd
from pyproj import Geod
from pathlib import Path
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
from rasterio.io import MemoryFile
from scipy.interpolate import interp1d
from tqdm import trange

## centerline_mesh

In [ ]:
@dataclass
class IntervalMeshResult:
    mesh: firedrake.IntervalMesh
    x: np.ndarray
    y: np.ndarray
    glacier_length: float
    mesh_length: float

def centerline_mesh(**kwargs):
    rgiid = kwargs.get('rgiid', '15-09534')
    centerline_path = kwargs['centerline_path']
    outline_path = kwargs['outline_path']
    extra_length = kwargs.get('extra_length', 0)
    n_cells = kwargs['n_cells']

    outlines = gpd.read_file(outline_path)
    centerlines = gpd.read_file(centerline_path)
    outline = outlines[outlines['rgi_id'].str.contains(rgiid)].geometry.values[0]
    centerline = centerlines[centerlines.intersects(outline)].geometry.values[0]

    geod = Geod(ellps = 'WGS84')
    x, y = centerline.xy
    distances = [0] + [geod.inv(x[i], y[i], x[i+1], y[i+1])[2] for i in range(len(x) - 1)]
    glacier_length = np.sum(distances)
    length = glacier_length + extra_length

    if extra_length > 0:
        azimuth, _, _ = geod.inv(x[-2], y[-2], x[-1], y[-1])
        x_new, y_new, _ = geod.fwd(x[-1], y[-1], azimuth, extra_length)
        x = np.append(x, x_new)
        y = np.append(y, y_new)
        distances.append(extra_length)

    mesh = firedrake.IntervalMesh(n_cells, length)

    return IntervalMeshResult(
        mesh = mesh,
        x = np.array(x),
        y = np.array(y),
        glacier_length = glacier_length,
        mesh_length = float(length)
    )

## map_to_mesh

In [ ]:
@dataclass
class InterpolateResult:
    data: firedrake.Function
    last_nonzero_value: float

def map_to_mesh(**kwargs):
    mesh = kwargs['mesh']
    data_path = kwargs['data_path']
    extension = Path(data_path).suffix
    dimension = kwargs.get('dimension', 1)
    element = kwargs.get('element', 'CG')
    ice_free_value = kwargs.get('ice_free_value', 0)
    key_value = kwargs.get('key_value', 'n/a')
    data_value = kwargs.get('data_value', 'n/a')
    key_dataset = kwargs.get('key_dataset', 'n/a')
    projection = kwargs.get('projection', 'EPSG:4326')

    if extension == '.tif':
        x, y = mesh.x, mesh.y

        with rasterio.open(data_path) as src:
            src_crs = src.crs
            target_crs = CRS.from_string(projection)

            if src_crs != target_crs:
                print(f'Reprojecting {data_path} from {src_crs} to {target_crs}')
                transform, width, height = calculate_default_transform(
                    src_crs, target_crs, src.width, src.height, *src.bounds)

                meta = src.meta.copy()
                meta.update({
                    'crs': target_crs,
                    'transform': transform,
                    'width': width,
                    'height': height
                })

                with MemoryFile() as memfile:
                    with memfile.open(**meta) as dst:
                        for i in range(1, src.count + 1):
                            reproject(
                                source = rasterio.band(src, i),
                                destination = rasterio.band(dst, i),
                                src_transform = src.transform,
                                src_crs = src_crs,
                                dst_transform = transform,
                                dst_crs = target_crs,
                                resampling = Resampling.bilinear
                            )
                    with memfile.open() as reproj:
                        values = np.array(list(reproj.sample(zip(x, y)))).flatten()
            else:
                values = np.array(list(src.sample(zip(x, y)))).flatten()

        distances = np.insert(np.cumsum([Geod(ellps = 'WGS84').inv(x[i], y[i], x[i+1], y[i+1])[2] for i in range(len(x) - 1)]), 0, 0)
        vertex_coords = mesh.mesh.coordinates.dat.data_ro.flatten()
        interp_vals = interp1d(distances, values, bounds_error = False, fill_value = 'extrapolate')(vertex_coords)
        interp_vals[vertex_coords > mesh.glacier_length] = ice_free_value

        V = firedrake.FunctionSpace(mesh.mesh, element, dimension)
        data_function = firedrake.Function(V)
        data_function.dat.data[:] = interp_vals
        last_nonzero = next((v for v in reversed(interp_vals) if v != 0), 0)

        return InterpolateResult(data = data_function, last_nonzero_value = float(last_nonzero))

    elif extension == '.csv':
        data = pd.read_csv(data_path)
        interp_func = interp1d(data[key_value], data[data_value], bounds_error = False, fill_value = 'extrapolate')

        vertex_keys = key_dataset.dat.data_ro
        data_on_mesh = interp_func(vertex_keys)

        V = firedrake.FunctionSpace(mesh.mesh, element, dimension)
        data_function = firedrake.Function(V)
        data_function.dat.data[:] = data_on_mesh

        return InterpolateResult(data = data_function, last_nonzero_value = np.nan)

## extend_to_mesh

In [ ]:
def extend_to_mesh(**kwargs):
    source_function = kwargs['source_function']
    source_mesh = kwargs['source_mesh']
    target_mesh = kwargs['target_mesh']
    ice_free_value = kwargs.get('ice_free_value', 0)

    # Step 1: Get distances along source mesh from vertex coordinates
    coords_src = source_mesh.mesh.coordinates.dat.data_ro[:]
    distances_src = np.insert(np.cumsum([
        np.linalg.norm(coords_src[i + 1] - coords_src[i])
        for i in range(len(coords_src) - 1)
    ]), 0, 0)

    values_src = source_function.dat.data_ro[:]
    assert len(distances_src) == len(values_src), "Mismatch in source distance and value lengths"

    # Step 2: Get distances along target mesh
    coords_tgt = target_mesh.mesh.coordinates.dat.data_ro[:]
    distances_tgt = np.insert(np.cumsum([
        np.linalg.norm(coords_tgt[i + 1] - coords_tgt[i])
        for i in range(len(coords_tgt) - 1)
    ]), 0, 0)

    # Step 3: Interpolate and apply cutoff
    interp_func = interp1d(distances_src, values_src, bounds_error=False, fill_value='extrapolate')
    values_tgt = interp_func(distances_tgt)
    values_tgt[distances_tgt > source_mesh.glacier_length] = ice_free_value

    # Step 4: Create new function on target mesh
    V_new = firedrake.FunctionSpace(target_mesh.mesh, source_function.function_space().ufl_element())
    f_new = firedrake.Function(V_new)
    f_new.dat.data[:] = values_tgt

    last_nonzero = next((v for v in reversed(values_tgt) if v != 0), 0)

    return InterpolateResult(data=f_new, last_nonzero_value=float(last_nonzero))


## solve_bed

In [ ]:
@dataclass
class InversionResult:
    bed: firedrake.Function
    misfits: list
    bed_evolution: list
    surface_evolution: list
    velocity_evolution: list

def solve_bed(**kwargs):
    K = kwargs['K']
    num_iterations = kwargs['num_iterations']
    Δt = kwargs['timestep']
    num_years = kwargs['model_time']
    s_ref = kwargs['surface']                        
    H_guess = kwargs['thickness_guess']
    u_guess = kwargs['velocity']                     
    a = kwargs['accumulation'] 
    model = kwargs['model']
    solver = kwargs['solver']
    mesh = kwargs['mesh']
    A = kwargs['fluidity']
        
    Q = s_ref.function_space()
    glacier_length = mesh.glacier_length
    X = np.arange(0, glacier_length, 1)

    bed_guess = firedrake.Function(Q).project(s_ref - H_guess)

    misfits = []
    bed_evolution = [np.array(bed_guess.at(X, tolerance = 1e-10))]
    surface_evolution = []
    velocity_evolution = []

    num_timesteps = int(num_years / Δt)

    bed_correction = firedrake.Function(Q)
    surface_misfit = firedrake.Function(Q)

    for iteration in trange(num_iterations):
        bed_mod = bed_guess.copy(deepcopy = True)
        H_mod = firedrake.Function(Q).project(s_ref - bed_mod)
        u_mod = u_guess.data.copy(deepcopy = True)
        s_mod = s_ref.copy(deepcopy = True)

        for step in range(num_timesteps):
            u_mod = solver.diagnostic_solve(
                velocity = u_mod, thickness = H_mod, surface = s_mod, fluidity = A
            )
            H_mod = solver.prognostic_solve(
                Δt, thickness = H_mod, velocity = u_mod,
                thickness_inflow = H_mod, accumulation = a
            )
            s_mod.project(bed_mod + H_mod)

        surface_misfit.project(s_mod - s_ref)
        bed_correction.project(-K * surface_misfit)
        bed_guess.project(bed_mod + bed_correction)

        misfits.append(np.linalg.norm(np.array(surface_misfit.at(X, tolerance = 1e-10))))
        bed_evolution.append(np.array(bed_guess.at(X, tolerance = 1e-10)))
        surface_evolution.append(np.array(s_mod.at(X, tolerance = 1e-10)))
        velocity_evolution.append(np.array(u_mod.at(X, tolerance = 1e-10)))

    return InversionResult(
        bed = bed_guess,
        misfits = misfits,
        bed_evolution = bed_evolution,
        surface_evolution = surface_evolution,
        velocity_evolution = velocity_evolution
    )

In [ ]:
# !jupyter nbconvert --to script centerflow.ipynb